<a href="https://colab.research.google.com/github/blockchainrelativity/Colab_Experiments/blob/main/image_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook is to be a playground for testing AI models, tensoryflow pipelines, training methods, and data structures to take in an image and turn it into a website.

# Setup

In [ ]:
OPENAI_API_KEY = 'sk-'
PINECONE_API_KEY = ''
PINECONE_API_ENV = ''
HG_API_KEY = ''

### Google Drive for imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

*install the necessary libraries and packages for your AI model. For example, if you are using a DCGAN model, you will need to install TensorFlow and Keras.*

In [ ]:
!pip install tensorflow
!pip install keras

*Once you have installed the necessary libraries, you can load your AI model. This will likely involve loading weights that were trained on a large dataset of code and images. For example, if you are using a pre-trained DCGAN model*

In [ ]:
from keras.models import load_model
generator = load_model('path/to/your/model.h5')

example code that uses a pre-trained DCGAN model to generate a website from an input image

In [ ]:
import cv2
import numpy as np

# Load the input image
input_image_path = 'path/to/your/image.jpg'
input_image = cv2.imread(input_image_path)
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
input_image = cv2.resize(input_image, (64, 64))

# Generate a website from the input image
generated_image = generator.predict(np.array([input_image])/255.0)[0]
generated_image = (generated_image * 255).astype(np.uint8)
generated_image = cv2.cvtColor(generated_image, cv2.COLOR_RGB2BGR)

# Output the website code
website_code = generate_website_code(generated_image)
print(website_code)


*save website to HTML file*

In [ ]:
with open('path/to/your/website.html', 'w') as f:
    f.write(website_code)


# Langchain setup

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

## Tools

In [ ]:
# Define which tools the agent can use to answer user queries
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

# CAMEL setup

In [ ]:
!pip install sentencepiece
!pip install langchain
!pip install huggingface_hub
!pip install transformers

# !pip install openai

In [ ]:
# from google.colab import drive, files
# from google.colab.patches import cv2_imshow
# from IPython.display import display, Javascript

# # # Mount Google Drive
# drive.mount('/content/drive')

# # Show file picker widget
# display(Javascript('''
#   google.colab.filesystem.inputFiles([
#     {
#       'name': 'Select a file',
#       'multiple': False
#     }
#   ], function(file) {
#     console.log('Selected file:', file);
#     google.colab.kernel.invokeFunction('notebook.run', [file])
#   });
# '''))

# # Define function to run when file is selected
# def run(file):
#   # Read contents of file
#   with open(file, 'r') as f:
#     content = f.read()

#   # Use contents as input for CAMEL agent
#   # ...


<IPython.core.display.Javascript object>

In [ ]:
# Image object classification model setup
!pip install pillow

import keras

from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16
import numpy as np
import argparse
import cv2

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to the input image")
args = vars(ap.parse_args())
# load the original image via OpenCV so we can draw on it and display
# it to our screen later
orig = cv2.imread(args["image"])


print("[INFO] loading and preprocessing image...")
image = image_utils.load_img(args["image"], target_size=(224, 224))
image = image_utils.img_to_array(image)


# our image is now represented by a NumPy array of shape (224, 224, 3),
# assuming TensorFlow "channels last" ordering of course, but we need
# to expand the dimensions to be (1, 3, 224, 224) so we can pass it
# through the network -- we'll also preprocess the image by subtracting
# the mean RGB pixel intensity from the ImageNet dataset
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


usage: ipykernel_launcher.py [-h] -i IMAGE
ipykernel_launcher.py: error: the following arguments are required: -i/--image


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# load the VGG16 network pre-trained on the ImageNet dataset
print("[INFO] loading network...")
model = VGG16(weights="imagenet")

# classify the image
print("[INFO] classifying image...")
preds = model.predict(image)
P = decode_predictions(preds)

# loop over the predictions and display the rank-5 predictions +
# probabilities to our terminal
for (i, (imagenetID, label, prob)) in enumerate(P[0]):
	print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))

# load the image via OpenCV, draw the top prediction on the image,
# and display the image to our screen
orig = cv2.imread(args["image"])
(imagenetID, label, prob) = P[0][0]
cv2.putText(orig, "Label: {}, {:.2f}%".format(label, prob * 100),
	(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
cv2.imshow("Classification", orig)
cv2.waitKey(0)

In [ ]:
from google.colab import files, drive
from PIL import Image
import numpy as np
from langchain.document_loaders.image import UnstructuredImageLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import sentencepiece

# Setup for HuggingFace Model
tokenizer = AutoTokenizer.from_pretrained("chainyo/alpaca-lora-7b")
model = AutoModelForCausalLM.from_pretrained("chainyo/alpaca-lora-7b")
# tokenizer = LlamaTokenizerFast.from_pretrained("anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g")
# model = LlamaForCausalLM.from_pretrained("anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g", from_pt=True)

# Upload file to Colab notebook
uploaded = files.upload()

# Get file name and contents
filename = list(uploaded.keys())[0]
file_content = uploaded[filename]


def process_image(image_path):
    image = Image.open(image_path).convert('RGB')
    # do any necessary image processing here
    image_array = np.array(image)
    rgb_vector = image_array.reshape(-1, 3)
    return rgb_vector

def run(image_path):
    rgb_vector = process_image(image_path)
    input_tokens = ["[CLS]"] + [" ".join(map(str, pixel)) for pixel in rgb_vector.tolist()] + ["[SEP]"]
    input_ids = tokenizer.encode(input_tokens, add_special_tokens=False, return_tensors="pt")
    output = model.generate(input_ids)
    return output

######################################

# def run(image_path):
#     rgb_vector = process_image(image_path)
#     llm_input = ','.join([str(p) for p in rgb_vector])
#     response = openai.Completion.create(
#         engine="text-davinci-002",
#         prompt=llm_input,
#         max_tokens=1024,
#         n=1,
#         stop=None,
#         temperature=0.5,
#     )
#     return response.choices[0].text
###############################################

# def file_to_bytes(file_path):
#     with open(file_path, "rb") as file:
#         file_content = file.read()
#     return file_content

########################################

# # load the image from the file and convert it to a binary format
# with Image.open(file_content + list(files.uploaded.keys())[0]) as img:
#     img_bin = img.tobytes()

########################################
# # define function to upload files
# def upload_file():
#     uploaded = files.upload()
#     for name, data in uploaded.items():
#         with open(name, 'wb') as f:
#             f.write(data)
#     print('File uploaded')

# # call the upload_file() function to upload the file
# upload_file()

# # load the image from the file and convert it to a binary format
# with Image.open('/content/' + list(files.uploaded.keys())[0]) as img:
#     img_bin = img.tobytes()

# Use contents as input for CAMEL agent
# ...


In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HG_API_KEY

In [ ]:
from pydantic import BaseModel
from typing import List, Union
# from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceHub
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

In [ ]:
#

In [ ]:
class ImageMessage(BaseModel):
    content: bytes

class CAMELAgent:

    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages


    def step(
        self,
        input_content: HumanMessage,
        image_content: bytes,
    ) -> AIMessage:


        image_message = ImageMessage(content=image_content, file_type="png")
        messages = self.update_messages(input_content)
        messages.append(image_message)

        output_message = self.model(messages)
        self.update_messages(output_message)

        return output_message

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# alternatively use this
# %env OPENAI_API_KEY=OPENAI_API_KEY

assistant_role_name = "Front End Developer"
user_role_name = "Project Manager"
task = "Develop a dashboard website that matches the image, if image has no color scheme or imagery, apply relevant themes to the design. Final task is to produce an HTML file to review the results"
word_limit = 50 # word limit for task brainstorming

In [ ]:
# Need to pick a model that can take in images
llm = HuggingFaceHub(
    repo_id="anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g",
    model_kwargs={"temperature":0.2}
)
task_llm = HuggingFaceHub(
    repo_id="anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g",
    model_kwargs={"temperature":1.0}
)

In [ ]:
task_specifier_sys_msg = SystemMessage(content="You can make a task more specific.")
task_specifier_prompt = (
"""Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
Please make it more specific. Be creative and imaginative.
Please reply with the specified task in {word_limit} words or less. Be specific and concise. Do not add anything else."""
)
task_specifier_template = HumanMessagePromptTemplate.from_template(template=task_specifier_prompt)
task_specify_agent = CAMELAgent(task_specifier_sys_msg, task_llm)
task_specifier_msg = task_specifier_template.format_messages(assistant_role_name=assistant_role_name,
                                                             user_role_name=user_role_name,
                                                             task=task, word_limit=word_limit)[0]

specified_task_msg = task_specify_agent.step(task_specifier_msg, file_content)
print(f"Specified task: {specified_task_msg.content}")
specified_task = specified_task_msg.content

if specified_task_msgs:
    specified_task_msg = specified_task_msgs[0]
else:
    # Handle the case where no message is generated
    specified_task_msg = HumanMessage(text="")

In [ ]:
specified_task_msg = task_specify_agent.step(task_specifier_msg)
print(f"Specified task: {specified_task_msg.content}")
specified_task = specified_task_msg.content

In [ ]:
assistant_inception_prompt = (
"""Never forget you are a {assistant_role_name} and I am a {user_role_name}. Never flip roles! Never instruct me!
We share a common interest in collaborating to successfully complete a task.
You must help me to complete the task.
Here is the task: {task}. Never forget our task!
I must instruct you based on your expertise and my needs to complete the task.

I must give you one instruction at a time.
You must write a specific solution that appropriately completes the requested instruction.
You must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.
Do not add anything else other than your solution to my instruction.
You are never supposed to ask me any questions you only answer questions.
You are never supposed to reply with a flake solution. Explain your solutions.
Your solution must be declarative sentences and simple present tense.
Unless I say the task is completed, you should always start with:

Solution: <YOUR_SOLUTION>

<YOUR_SOLUTION> should be specific and provide preferable implementations and examples for task-solving.
Always end <YOUR_SOLUTION> with: Next request."""
)

user_inception_prompt = (
"""Never forget you are a {user_role_name} and I am a {assistant_role_name}. Never flip roles! You will always instruct me.
We share a common interest in collaborating to successfully complete a task.
I must help you to complete the task.
Here is the task: {task}. Never forget our task!
You must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:

1. Instruct with a necessary input:
Instruction: <YOUR_INSTRUCTION>
Input: <YOUR_INPUT>

2. Instruct without any input:
Instruction: <YOUR_INSTRUCTION>
Input: None

The "Instruction" describes a task or question. The paired "Input" provides further context or information for the requested "Instruction".

You must give me one instruction at a time.
I must write a response that appropriately completes the requested instruction.
I must decline your instruction honestly if I cannot perform the instruction due to physical, moral, legal reasons or my capability and explain the reasons.
You should instruct me not ask me questions.
Now you must start to instruct me using the two ways described above.
Do not add anything else other than your instruction and the optional corresponding input!
Keep giving me instructions and necessary inputs until you think the task is completed.
When the task is completed, you must only reply with a single word <CAMEL_TASK_DONE>.
Never say <CAMEL_TASK_DONE> unless my responses have solved your task."""
)

In [ ]:
def get_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):

    assistant_sys_template = SystemMessagePromptTemplate.from_template(template=assistant_inception_prompt)
    assistant_sys_msg = assistant_sys_template.format_messages(assistant_role_name=assistant_role_name, user_role_name=user_role_name, task=task)[0]

    user_sys_template = SystemMessagePromptTemplate.from_template(template=user_inception_prompt)
    user_sys_msg = user_sys_template.format_messages(assistant_role_name=assistant_role_name, user_role_name=user_role_name, task=task)[0]

    return assistant_sys_msg, user_sys_msg

In [ ]:
assistant_sys_msg, user_sys_msg = get_sys_msgs(assistant_role_name, user_role_name, specified_task)
assistant_agent = CAMELAgent(assistant_sys_msg, llm)
user_agent = CAMELAgent(user_sys_msg, llm)

# Reset agents
assistant_agent.reset()
user_agent.reset()

# Initialize chats
assistant_msg = HumanMessage(
    content=(f"{user_sys_msg.content}. "
                "Now start to give me introductions one by one. "
                "Only reply with Instruction and Input."))

user_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
user_msg = assistant_agent.step(user_msg)

In [ ]:
print(f"Original task prompt:\n{task}\n")
print(f"Specified task prompt:\n{specified_task}\n")

chat_turn_limit, n = 30, 0
while n < chat_turn_limit:
    n += 1
    user_ai_msg = user_agent.step(assistant_msg)
    user_msg = HumanMessage(content=user_ai_msg.content)
    print(f"AI User ({user_role_name}):\n\n{user_msg.content}\n\n")

    assistant_ai_msg = assistant_agent.step(user_msg)
    assistant_msg = HumanMessage(content=assistant_ai_msg.content)
    print(f"AI Assistant ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
    if "<CAMEL_TASK_DONE>" in user_msg.content:
        break

# HiAGI Setup

In [ ]:
!git clone https://github.com/blockchainrelativity/HiAGI.git
!cd HiAGI

Cloning into 'HiAGI'...
remote: Enumerating objects: 931, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 931 (delta 121), reused 113 (delta 107), pack-reused 759
Receiving objects: 100% (931/931), 327.48 KiB | 2.85 MiB/s, done.
Resolving deltas: 100% (558/558), done.


In [ ]:
!pip install -r HiAGI/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
!python -m spacy download en_core_web_sm

2023-05-04 06:09:35.579252: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 06:09:36.995175: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!python HiAGI/main.py

python3: can't open file '/content/main.py': [Errno 2] No such file or directory
